In [15]:
# Importing libraries
from youtube_transcript_api import YouTubeTranscriptApi
from pymongo import MongoClient
import requests
from clearml import Task, PipelineController
# from sentence_transformers import SentenceTransformer
from qdrant_client import QdrantClient
from qdrant_client.http import models
from sentence_transformers import SentenceTransformer
import re

In [16]:
# Task.init(project_name="Featurization Pipeline", task_name="Featurization Pipeline")

In [17]:
from pymongo import MongoClient

# Connect to the MongoDB server
client = MongoClient("mongodb://localhost:27017/")  # Replace with your connection string

# Access the database and collection
db = client["media_data_final1"]
collection = db["raw_data_final"]

# Retrieve all records
records = collection.find()
count = collection.count_documents({})
print(count)
# Iterate through and print each record
# for record in records:
#     print(record)


2583


In [18]:
# Connecting to MongoDB

# MongoDB Configuration
MONGO_URI = "mongodb://localhost:27017/"
DB_NAME = "media_data_final1"
EMBEDDINGS_COLLECTION_NAME = "embeddings"
RAW_DATA_COLLECTION_NAME = "raw_data_final"

# Connect to MongoDB
mongo_client = MongoClient(MONGO_URI)
db = mongo_client[DB_NAME]
collection = db[RAW_DATA_COLLECTION_NAME]

In [19]:
# Connecting to Qdrant

qdrant_client = QdrantClient(":memory:")
collection_name = "text_collection"
qdrant_client.recreate_collection(
    collection_name=collection_name,
    vectors_config=models.VectorParams(size=384, distance="Cosine")
)

C:\Users\harsh\AppData\Local\Temp\ipykernel_2260\4197240320.py:5: DeprecationWarning: `recreate_collection` method is deprecated and will be removed in the future. Use `collection_exists` to check collection existence and `create_collection` instead.
  qdrant_client.recreate_collection(


True

In [9]:
def create_embeddings_and_persist(data, source, url):
    mongo_client = MongoClient(MONGO_URI)
    db = mongo_client[DB_NAME]
    collection = db[EMBEDDINGS_COLLECTION_NAME]
    model = SentenceTransformer('paraphrase-MiniLM-L6-v2') 
    embedding = model.encode(data)
    
    document = {
        "source": source,
        "url": url,
        "embedding": embedding.tolist(),
        "data": data
    }
    collection.insert_one(document)
    print("Embeddings stored into MongoDB")

In [10]:
def chunk_youtube_data(data):
    sentences = [i['text'] for i in data]
    chunks = " ".join(sentences)
    return chunks

In [ ]:
def create_data_embeddings():
    client = MongoClient("mongodb://localhost:27017/") 

    # Access the database and collection
    db = client["media_data_final1"]
    collection = db["raw_data_final"]

    # Retrieve all records
    records = collection.find()
    count = 0
    for record in records:
        count+=1
        source = record.get("source")
        if source == "web":
            continue
            data = record.get("content")
        data = record.get("data")
        if (source == "youtube"):
            data = chunk_youtube_data(record.get("data"))
        create_embeddings_and_persist(data, source, record.get("url"))
        

In [12]:
create_data_embeddings()

Embeddings stored into MongoDB
Embeddings stored into MongoDB
Embeddings stored into MongoDB
Embeddings stored into MongoDB
Embeddings stored into MongoDB
Embeddings stored into MongoDB
Embeddings stored into MongoDB
Embeddings stored into MongoDB
Embeddings stored into MongoDB
Embeddings stored into MongoDB
Embeddings stored into MongoDB
Embeddings stored into MongoDB
Embeddings stored into MongoDB
Embeddings stored into MongoDB
Embeddings stored into MongoDB
Embeddings stored into MongoDB
Embeddings stored into MongoDB
Embeddings stored into MongoDB
Embeddings stored into MongoDB
Embeddings stored into MongoDB
Embeddings stored into MongoDB
Embeddings stored into MongoDB
Embeddings stored into MongoDB
Embeddings stored into MongoDB
Embeddings stored into MongoDB
Embeddings stored into MongoDB
Embeddings stored into MongoDB
Embeddings stored into MongoDB
Embeddings stored into MongoDB
Embeddings stored into MongoDB
Embeddings stored into MongoDB
Embeddings stored into MongoDB
Embeddin

In [13]:
def clean_text(text):
    text = re.sub(r'<.*?>', '', text)  # Remove HTML tags
    text = re.sub(r'[^a-zA-Z0-9\s]', '', text)  # Remove special characters
    return text.strip().lower()

def featurize_web_data():
    mongo_client = MongoClient(MONGO_URI)
    db = mongo_client[DB_NAME]
    embedding_collection = db[EMBEDDINGS_COLLECTION_NAME]
    model = SentenceTransformer('paraphrase-MiniLM-L6-v2')
    web_data = collection.find({"source": "web"})
    for doc in web_data:
        sentences = doc['content'].split('\n')
        sentences = [clean_text(sentence) for sentence in sentences]
        paragraph = " ".join(sentences)
        paragraph = re.sub(r'\s+', ' ', paragraph)
        paragraph = doc['title'] + "\n" + paragraph
        model = SentenceTransformer('paraphrase-MiniLM-L6-v2')
        embedding = model.encode(paragraph)
        document = {
            "source": "web",
            "url": doc.get("url", ''),
            "embedding": embedding.tolist(),
            "data": paragraph
        }
        embedding_collection.insert_one(document)
        print(f"Saved embedding data into MongoDB for URL: {doc.get('url', '')}")

In [14]:
featurize_web_data()

Saved embedding data into MongoDB for URL: https://docs.ros.org/en/galactic/Installation.html
Saved embedding data into MongoDB for URL: https://docs.ros.org/en/galactic/Installation/Ubuntu-Install-Debians.html
Saved embedding data into MongoDB for URL: https://docs.ros.org/en/galactic/Installation/Windows-Install-Binary.html
Saved embedding data into MongoDB for URL: https://docs.ros.org/en/galactic/Installation/RHEL-Install-RPMs.html
Saved embedding data into MongoDB for URL: https://docs.ros.org/en/galactic/Installation/Alternatives.html
Saved embedding data into MongoDB for URL: https://docs.ros.org/en/galactic/Installation/Alternatives/Ubuntu-Development-Setup.html
Saved embedding data into MongoDB for URL: https://docs.ros.org/en/galactic/Installation/Alternatives/Ubuntu-Install-Binary.html
Saved embedding data into MongoDB for URL: https://docs.ros.org/en/galactic/Installation/Alternatives/Windows-Development-Setup.html
Saved embedding data into MongoDB for URL: https://docs.ros

In [20]:
qdrant = QdrantClient(":memory:")

In [21]:
def insert_embeddings_to_qdrant(collection_name):
    mongo_client = MongoClient(MONGO_URI)
    db = mongo_client[DB_NAME]
    embedding_collection = db[EMBEDDINGS_COLLECTION_NAME]

    
    my_collection = collection_name
    qdrant.create_collection(
    collection_name=my_collection,
    vectors_config=models.VectorParams(size=384, distance=models.Distance.COSINE)
    )
    web_data = embedding_collection.find({})
    for i, doc in enumerate(web_data):

        embedding = doc['embedding']
        source = doc['source']
        url = doc['url']
        data = doc['data']
        qdrant.upsert(
            collection_name= my_collection,
            points=[models.PointStruct(
                id=i,
                vector=embedding,
                payload={"data": data, "source": source, "url": url}
            )]
        )
    print("Inserted all embeddings into Qdrant")


        
    
    

In [22]:
insert_embeddings_to_qdrant("text_collection")

Inserted all embeddings into Qdrant


In [23]:

qdrant.count(
    collection_name="text_collection",
    exact=True,
)

CountResult(count=2583)

# Inference


In [24]:
import random
from tqdm import tqdm
import textwrap

from youtube_transcript_api import YouTubeTranscriptApi

from pymongo import MongoClient

from sentence_transformers import SentenceTransformer #TODO: 90% sure this is the best for embedding in our case, just check once

from qdrant_client import QdrantClient
from qdrant_client.http import models
from torchvision.transforms import Resize

import torch

from peft import PeftModel
from huggingface_hub import login
from transformers import AutoTokenizer, AutoModelForCausalLM
from transformers.utils import is_flash_attn_2_available 
from transformers import BitsAndBytesConfig

In [25]:
#LLM

login() # Login to huggingface # TODO: Finalize whether this is required when you finalize the model

use_quantization_config = True
# model_id = "TheBloke/CapybaraHermes-2.5-Mistral-7B-GPTQ"
# Load local finetuned model

#Instantiate tokenizer (tokenizer turns text into numbers ready for the model) 
tokenizer = AutoTokenizer.from_pretrained("TheBloke/CapybaraHermes-2.5-Mistral-7B-GPTQ", use_fast=True)

# Instantiate the model
base_model = AutoModelForCausalLM.from_pretrained("TheBloke/CapybaraHermes-2.5-Mistral-7B-GPTQ")
llm_model = PeftModel.from_pretrained(base_model, "neo2446/AI_Project_RAG_ROS_Finetuned_2.5-Mistral-7B-GPTQ")
llm_model.to("cuda")
llm_model.eval() # put model in evaluation mode

`low_cpu_mem_usage` was None, now default to True since model is quantized.
c:\Users\harsh\anaconda3\envs\ai\lib\site-packages\transformers\modeling_utils.py:5006: FutureWarning: `_is_quantized_training_enabled` is going to be deprecated in transformers 4.39.0. Please use `model.hf_quantizer.is_trainable` instead
  warnings.warn(
Some weights of the model checkpoint at TheBloke/CapybaraHermes-2.5-Mistral-7B-GPTQ were not used when initializing MistralForCausalLM: ['model.layers.0.mlp.down_proj.bias', 'model.layers.0.mlp.gate_proj.bias', 'model.layers.0.mlp.up_proj.bias', 'model.layers.0.self_attn.k_proj.bias', 'model.layers.0.self_attn.o_proj.bias', 'model.layers.0.self_attn.q_proj.bias', 'model.layers.0.self_attn.v_proj.bias', 'model.layers.1.mlp.down_proj.bias', 'model.layers.1.mlp.gate_proj.bias', 'model.layers.1.mlp.up_proj.bias', 'model.layers.1.self_attn.k_proj.bias', 'model.layers.1.self_attn.o_proj.bias', 'model.layers.1.self_attn.q_proj.bias', 'model.layers.1.self_attn.v_proj.

adapter_config.json:   0%|          | 0.00/695 [00:00<?, ?B/s]

c:\Users\harsh\anaconda3\envs\ai\lib\site-packages\huggingface_hub\file_download.py:139: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\harsh\.cache\huggingface\hub\models--neo2446--AI_Project_RAG_ROS_Finetuned_2.5-Mistral-7B-GPTQ. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


adapter_model.safetensors:   0%|          | 0.00/13.6M [00:00<?, ?B/s]

PeftModelForCausalLM(
  (base_model): LoraModel(
    (model): MistralForCausalLM(
      (model): MistralModel(
        (embed_tokens): Embedding(32002, 4096, padding_idx=0)
        (layers): ModuleList(
          (0-31): 32 x MistralDecoderLayer(
            (self_attn): MistralSdpaAttention(
              (rotary_emb): MistralRotaryEmbedding()
              (k_proj): QuantLinear()
              (o_proj): QuantLinear()
              (q_proj): lora.QuantLinear(
                (base_layer): QuantLinear()
                (lora_dropout): ModuleDict(
                  (default): Dropout(p=0.1, inplace=False)
                )
                (lora_A): ModuleDict(
                  (default): Linear(in_features=4096, out_features=8, bias=False)
                )
                (lora_B): ModuleDict(
                  (default): Linear(in_features=8, out_features=4096, bias=False)
                )
                (lora_embedding_A): ParameterDict()
                (lora_embedding_B): Parame

In [26]:
# ROS questions generated with GPT4
gpt4_questions = [
    "What are the main differences between ROS 1 and ROS 2?",
    "Explain the concept of topics, services, and actions in ROS.",
    "What is the purpose of the catkin_make command in ROS 1?",
    "What is the function of rviz in ROS?",
    "What is the purpose of rqt_graph, and how can it help in debugging a ROS system?"
] 

# Manually created question list
manual_questions = [
    "What is ros?",
    "Tell me how can I navigate to a specific pose - include replanning aspects in your answer",
    "Can you provide me with code for this task? Task:Tell me how can I navigate to a specific pose"
]

query_list = gpt4_questions + manual_questions

In [27]:
def prompt_formatter(query: str, 
                     context_items: list[dict]) -> str:
    """
    Augments query with text-based context from context_items.
    """
    # Join context items into one dotted paragraph
    context = "- " + "\n- ".join([item["sentence_chunk"] for item in context_items]) #TODO: This works, but maybe make it better?

    system_message = f"You are a robotic operating system (ROS) developer, using given context as additional information and answer the query, just your answer explanatory answer would suffice. Here is the context:{context}." # For out task we can use this as a system message

    # Default prompt template
    prompt=f'''<|im_start|>system
    {system_message}<|im_end|>
    <|im_start|>user
    {query}<|im_end|>
    <|im_start|>assistant
    '''
    return prompt

def ask(query, 
        temperature=0.7,
        top_p=0.95,
        top_k=40,
        max_new_tokens=512,
        format_answer_text=True, 
        return_answer_only=True):
    """
    Takes a query, finds relevant resources/context and generates an answer to the query based on the relevant resources.
    """
    # Connect to Qdrant 
    
    my_collection = "text_collection" # Collection name in Qdrant
    model = SentenceTransformer('paraphrase-MiniLM-L6-v2')

    search_results = qdrant.search( 
    collection_name=my_collection,
    query_vector=model.encode(query),
    limit=5
    )
    
    # Create a list of context items
    context_items = []
    for result in search_results:
        context_items.append({"sentence_chunk": result.payload['data']})

    scores = [result.score for result in search_results]

    # Add score to context item
    for i, item in enumerate(context_items):
        item["score"] = scores[i] # return score back to CPU 
        
    # Format the prompt with context items
    prompt = prompt_formatter(query=query,
                              context_items=context_items)
    
    # Tokenize the prompt
    input_ids = tokenizer(prompt, return_tensors="pt").to("cuda")

    # Generate an output of tokens
    outputs = llm_model.generate(**input_ids,
                                 temperature=temperature,
                                 do_sample=True,
                                 top_p=top_p,
                                 top_k=top_k,
                                 max_new_tokens=max_new_tokens)
    
    # Turn the output tokens into text
    output_text = tokenizer.decode(outputs[0])

    if format_answer_text:
        # Replace special tokens and unnecessary help message
        prompt_format_matched = prompt.replace('<|im_start|>', '<|im_start|> ').replace('<|im_end|>\n', '<|im_end|> \n')
        output_text = output_text.replace('<s>', '').replace(prompt_format_matched, '').replace('<|im_end|>', '')

    # Only return the answer without the context items
    if return_answer_only:
        return output_text, "No context items returned"
    
    return output_text, context_items

def print_wrapped(text, wrap_length=80):
    wrapped_text = textwrap.fill(text, wrap_length)
    print(wrapped_text)

In [28]:
## Testing out everything

query = random.choice(query_list)
print(f"Query: {query}")

# Answer query with context and return context 
answer, context_items = ask(query=query, 
                            temperature=0.7,
                            max_new_tokens=512,
                            return_answer_only=False)

print(f"Answer:\n")
print_wrapped(answer)
print(f"Context items:")
context_items

Query: What is the purpose of rqt_graph, and how can it help in debugging a ROS system?


Setting `pad_token_id` to `eos_token_id`:None for open-end generation.


Answer:

 rqt_graph is a graphical introspection tool in ROS 2 that helps visualize the
changing nodes and topics, as well as the connections between them. It serves as
a powerful debugging and troubleshooting aid in a ROS system by providing a
visual representation of the nodes and topics within the system.  The purpose of
rqt_graph is to help users:  1. Visualize the inter-relationships between nodes
and topics: It provides a graphical representation of the nodes and topics
within a ROS system, allowing users to quickly identify the nodes that publish
or subscribe to a specific topic, making it easier to trace the flow of data
within the system. 2. Identify communication patterns: rqt_graph allows users to
observe the connections between nodes and topics, which can be helpful in
understanding how different components interact with each other, especially in
complex systems. 3. Locate specific nodes and topics: With the graph's zoom-in
and zoom-out features, users can easily find the n

[{'sentence_chunk': 'Basic Concepts\nROS 2 is a middleware based on a strongly-typed, anonymous publish/subscribe mechanism that allows for message passing between different processes.\nAt the heart of any ROS 2 system is the ROS graph.\nThe ROS graph refers to the network of nodes in a ROS system and the connections between them by which they communicate.\nThese are the concepts that will help you get started understanding the basics of ROS 2.\n:maxdepth: 1\nBasic/About-Nodes\nBasic/About-Discovery\nBasic/About-Interfaces\nBasic/About-Topics\nBasic/About-Services\nBasic/About-Actions\nBasic/About-Parameters\nBasic/About-Command-Line-Tools\nBasic/About-Launch\nBasic/About-Client-Libraries',
  'score': 0.6384204626083374},
 {'sentence_chunk': 'Overview and usage of RQt — ROS 2 Documentation: Galactic  documentation\nwarning youre reading the documentation for a version of ros 2 that has reached its eol endoflife and is no longer officially supported if you want uptodate information plea

In [29]:
import gradio as gr

# Define the questions and their corresponding answers
# questions_answers = {
#     "What is AI?": "Artificial Intelligence (AI) is the simulation of human intelligence in machines programmed to think and act like humans.",
#     "How does machine learning work?": "Machine learning works by training algorithms on data to recognize patterns and make predictions or decisions without explicit programming. Random words to test if the words get wrapped to the next line.",
#     "What is deep learning?": "Deep learning is a subset of machine learning involving neural networks with many layers that learn hierarchical features from data.",
#     "What are neural networks?": "Neural networks are computational models inspired by the human brain, used to recognize patterns and solve complex problems.",
#     "What is the difference between supervised and unsupervised learning?": (
#         "Supervised learning involves labeled data to train models, "
#         "while unsupervised learning works with unlabeled data to find hidden patterns or structure."
#     )
# }
questions = query_list

# Function to fetch the answer for a selected question
def get_answer(question):
    return ask(question, temperature=0.7, max_new_tokens=512, return_answer_only=True)[0]

# Create the Gradio interface
with gr.Blocks() as app:
    gr.Markdown("# ROS2 Query App")
    gr.Markdown("### Select a question from the dropdown menu to get the answer")
    
    # Dropdown menu for questions
    question_dropdown = gr.Dropdown(
        label="Query",
        # choices=list(questions_answers.keys()),
        # value=list(questions_answers.keys())[0]
        choices=questions,
        value=questions[0]
    )
    # Button to trigger answer display
    submit_button = gr.Button("Submit")
    
    # Output text box for the answer
    answer_output = gr.Textbox(
        label="Answer",
        interactive=False
    )
    
    
    
    # Link dropdown and button to the function
    submit_button.click(get_answer, inputs=question_dropdown, outputs=answer_output)

# Launch the app
app.launch()



Running on local URL:  http://127.0.0.1:7860

To create a public link, set `share=True` in `launch()`.


Setting `pad_token_id` to `eos_token_id`:None for open-end generation.
